In [ ]:
import os
import pandas as pd
from urllib.parse import unquote
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'AppleGothic'  # macOS: 'AppleGothic'

# 현재 디렉토리의 모든 .csv 파일 리스트 가져오기
csv_files = [file for file in os.listdir('.') if file.endswith('.csv')]

# 각 파일 읽기
dataframes = []  # 배열로 변경
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append((file, df))  # 파일 이름과 데이터프레임을 튜플로 저장
        print(f"{file} 읽기 완료")
    except Exception as e:
        print(f"{file} 읽기 실패: {e}")

# 확인: 읽어온 데이터프레임 목록 출력
print(f"총 {len(dataframes)}개의 CSV 파일을 읽어왔습니다.")

In [ ]:
# 모두 225개의 row가 잘 들어있는지 확인
for file, df in dataframes:
    print(f"{len(df)} rows")

In [ ]:
def calculate_correct_answers(file_df_tuple):
    """
    데이터프레임에서 'answer'와 'user_choice' 열이 같은 값을 세는 함수.
    """
    file, df = file_df_tuple
    if 'answer' in df.columns and 'user_choice' in df.columns:  # 열 확인
        return (file, (df['answer'].str.lower() == df['user_choice'].str.lower()).sum())
    else:
        return (file, None)  # 열이 없으면 None 반환

# map을 사용하여 데이터프레임 리스트에 적용
results = map(calculate_correct_answers, dataframes)

# 결과를 리스트로 변환
results = list(results)

# count 기준으로 정렬 (None 값은 맨 뒤로 이동)
sorted_results = sorted(results, key=lambda x: (x[1] is None, x[1]), reverse=True)

# 결과 출력
for file, count in sorted_results:
    if count is not None:
        print(f"{file}: 정답 수 = {count}, 비율 = {int(count / 225 * 100)}%")
    else:
        print(f"{file}: 'answer' 또는 'user_choice' 열이 없습니다.")

#### plot ####

# 유효한 count 값들만 추출
valid_counts = [count for _, count in results if count is not None]

# valid_counts 값을 각각 225로 나누고 100을 곱한 새로운 리스트 생성
scaled_counts = [(count / 225) * 100 for count in valid_counts]

# Seaborn을 사용하여 scaled_counts 분포 시각화
plt.figure(figsize=(10, 6), dpi=200)
sns.histplot(scaled_counts, bins=20, kde=True)

# 그래프 제목과 레이블 설정
plt.title('정답률')
plt.xlabel('Scaled Number of Correct Answers (%)')
plt.ylabel('Frequency')

# 그래프 출력
plt.show()

In [ ]:
def calculate_correct_answers_and_total_by_difficulty(file_df_tuple):
    """
    데이터프레임에서 'answer'와 'user_choice' 열이 같은 값의 수와 총 응답 수를 'difficulty_of_choices' 별로 세는 함수.
    """
    file, df = file_df_tuple
    if 'answer' in df.columns and 'user_choice' in df.columns and 'difficulty_of_choices' in df.columns:  # 열 확인
        grouped = df.groupby('difficulty_of_choices').apply(
            lambda x: ((x['answer'].str.lower() == x['user_choice'].str.lower()).sum(), len(x))
        )
        return (file, grouped.to_dict())  # 그룹별 (정답 수, 전체 수)를 딕셔너리로 반환
    else:
        return (file, None)  # 열이 없으면 None 반환

# map을 사용하여 데이터프레임 리스트에 적용
results = map(calculate_correct_answers_and_total_by_difficulty, dataframes)

# 결과를 리스트로 변환
results = list(results)

#### 난이도별 정답 수 및 총 응답 수 누적 ####

from collections import defaultdict

# 난이도별 정답 수 및 총 응답 수를 누적할 딕셔너리 초기화
total_counts_by_difficulty = defaultdict(lambda: {"correct": 0, "total": 0})

# 결과를 누적하면서 출력하기
for file, counts in results:
    if counts is not None:
        # print(f"{file}:")
        for difficulty, (correct_count, total_count) in counts.items():
            # 현재 파일의 난이도별 정답 수와 총 응답 수를 출력
            accuracy = (correct_count / total_count * 100) if total_count > 0 else 0
            # print(f"  난이도 '{difficulty}': 정답 수 = {correct_count}, 전체 응답 수 = {total_count}, 비율 = {int(accuracy)}%")

            # 난이도별로 전체 정답 수와 총 응답 수 누적
            total_counts_by_difficulty[difficulty]["correct"] += correct_count
            total_counts_by_difficulty[difficulty]["total"] += total_count
    else:
        print(f"{file}: 'answer', 'user_choice' 또는 'difficulty_of_choices' 열이 없습니다.")

# 모든 파일을 합친 난이도별 결과 출력
print("\n전체 파일 난이도별 정답률 합계:")

# 시각화를 위한 데이터 준비
difficulties = []
accuracies = []

for difficulty, counts in total_counts_by_difficulty.items():
    total_correct = counts["correct"]
    total_total = counts["total"]
    total_accuracy = (total_correct / total_total * 100) if total_total > 0 else 0
    print(f"난이도 '{difficulty}': 전체 정답 수 = {total_correct}, 전체 응답 수 = {total_total}, 비율 = {int(total_accuracy)}%")

    # 시각화를 위한 데이터 저장
    difficulties.append(difficulty)
    accuracies.append(total_accuracy)

# 난이도별 데이터를 데이터프레임으로 변환
data = pd.DataFrame({
    'Difficulty': difficulties,
    'Accuracy': accuracies
})

# 숫자형 난이도로 변환 (선형 회귀를 위해)
data['Difficulty_Numeric'] = data['Difficulty'].rank(method='dense')  # 난이도 레이블을 숫자로 변환

# 선형 회귀선 추가
plt.figure(figsize=(10, 6), dpi=200)
sns.regplot(
    x='Difficulty_Numeric',
    y='Accuracy',
    data=data,
    scatter=True,
    line_kws={"color": "red"},  # 회귀선 색상 설정
    scatter_kws={"s": 50}       # 산점도 크기 설정
)

# 그래프 제목 및 레이블 설정
plt.title('수식의 난이도에 따른 정답률')
plt.xlabel('Difficulty Level (Numeric)')
plt.ylabel('Correct Answer Rate (%)')

# 난이도 레이블을 원래 텍스트로 대체
plt.xticks(data['Difficulty_Numeric'], data['Difficulty'], rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
dataframes[0][1]['round']

In [ ]:
def calculate_correct_answers_and_total_by_round(file_df_tuple):
    """
    데이터프레임에서 'answer'와 'user_choice' 열이 같은 값의 수와 총 응답 수를 'round'별로 세는 함수.
    여기서 'round'의 값은 '-' 앞의 숫자만 사용하여 그룹화.
    """
    file, df = file_df_tuple
    if 'answer' in df.columns and 'user_choice' in df.columns and 'round' in df.columns:  # 열 확인
        # 'round'에서 '-' 앞의 숫자만 추출
        df['round_group'] = df['round'].str.split('-').str[0]
        grouped = df.groupby('round_group').apply(
            lambda x: ((x['answer'].str.lower() == x['user_choice'].str.lower()).sum(), len(x))
        )
        return (file, grouped.to_dict())  # 그룹별 (정답 수, 전체 수)를 딕셔너리로 반환
    else:
        return (file, None)  # 열이 없으면 None 반환

# map을 사용하여 데이터프레임 리스트에 적용
results_by_round = map(calculate_correct_answers_and_total_by_round, dataframes)

# 결과를 리스트로 변환
results_by_round = list(results_by_round)
for e in results_by_round:
    print(e[0], end=": ")
    print(round(e[1]['1'][0] / e[1]['1'][1] * 100, 1), end=",\t")
    print(round(e[1]['2'][0] / e[1]['2'][1] * 100, 1), end=",\t")
    print(round(e[1]['3'][0] / e[1]['3'][1] * 100, 1))

#### round별 정답 수 및 총 응답 수 누적 ####

# round별 정답 수 및 총 응답 수를 누적할 딕셔너리 초기화
total_counts_by_round = defaultdict(lambda: {"correct": 0, "total": 0})

# 결과를 누적하면서 출력하기
for file, counts in results_by_round:
    if counts is not None:
        for round_group, (correct_count, total_count) in counts.items():
            # round별로 전체 정답 수와 총 응답 수 누적
            total_counts_by_round[round_group]["correct"] += correct_count
            total_counts_by_round[round_group]["total"] += total_count
    else:
        print(f"{file}: 'answer', 'user_choice' 또는 'round' 열이 없습니다.")

# 모든 파일을 합친 round별 결과 출력
print("\n전체 파일 round별 정답률 합계:")

# 시각화를 위한 데이터 준비
rounds = []
accuracies = []

for round_group, counts in total_counts_by_round.items():
    total_correct = counts["correct"]
    total_total = counts["total"]
    total_accuracy = (total_correct / total_total * 100) if total_total > 0 else 0
    print(f"Round '{round_group}': 전체 정답 수 = {total_correct}, 전체 응답 수 = {total_total}, 비율 = {int(total_accuracy)}%")

    # 시각화를 위한 데이터 저장
    rounds.append(round_group)
    accuracies.append(total_accuracy)

# round별 데이터를 데이터프레임으로 변환
data_round = pd.DataFrame({
    'Round': rounds,
    'Accuracy': accuracies
})

# 숫자형 round로 변환 (선형 회귀를 위해)
data_round['Round_Numeric'] = data_round['Round'].astype(int)

# 선형 회귀선 추가
plt.figure(figsize=(10, 6))
sns.regplot(
    x='Round_Numeric',
    y='Accuracy',
    data=data_round,
    scatter=True,
    line_kws={"color": "blue"},  # 회귀선 색상 설정
    scatter_kws={"s": 50}       # 산점도 크기 설정
)

# 그래프 제목 및 레이블 설정
plt.title('선택지의 속도에 따른 정답률')
plt.xlabel('Round (Numeric)')
plt.ylabel('Correct Answer Rate (%)')

plt.tight_layout()
plt.show()
